### Set up

In [ ]:
import json
import os
import azure.identity
import dotenv
import numpy as np
import openai
import pandas as pd

In [74]:
# Set up OpenAI client based on environment variables
dotenv.load_dotenv()
AZURE_OPENAI_SERVICE = os.getenv("AZURE_OPENAI_SERVICE")
AZURE_OPENAI_ADA_DEPLOYMENT = os.getenv("AZURE_OPENAI_ADA_DEPLOYMENT")


openai_client = openai.AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-15-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

def get_embedding(text):
    get_embeddings_response = openai_client.embeddings.create(model=AZURE_OPENAI_ADA_DEPLOYMENT, input=text)
    return get_embeddings_response.data[0].embedding
    
def get_embeddings(sentences):
    embeddings_response = openai_client.embeddings.create(model=AZURE_OPENAI_ADA_DEPLOYMENT, input=sentences)
    return [embedding_object.embedding for embedding_object in embeddings_response.data]

### Vector representations

In [75]:
vector = get_embedding("ข้าวต้มกุ้ง") 

In [76]:
len(vector)

3072

In [77]:
from tabulate import tabulate
# Convert the vector to a list of lists
vector_list = [[value] for value in vector]

# Create a table with headers
table = tabulate(vector_list, headers=["Vector Value"], tablefmt="pretty")

# Print the table
print(table)


+-------------------------+
|      Vector Value       |
+-------------------------+
|  -0.009463845752179623  |
|  -0.05069706216454506   |
|  -0.00126791896764189   |
|   0.03521614149212837   |
|  0.044985271990299225   |
|   0.0019129574066028    |
|  -0.03029218502342701   |
|  -0.017312632873654366  |
|  0.016997499391436577   |
| 0.00018295578774996102  |
|  -0.024284956976771355  |
|  0.011246317997574806   |
|  0.046009454876184464   |
|   0.04000222682952881   |
| -0.0077897002920508385  |
|   0.02412739023566246   |
|   0.01409236527979374   |
|   0.00790295097976923   |
|  -0.03819021210074425   |
|  0.034684352576732635   |
|  -0.009444150142371655  |
|  0.007765080314129591   |
|   0.04825478047132492   |
| -0.0008136839023791254  |
|  0.043448999524116516   |
| -0.0037520553451031446  |
|  -0.04916078969836235   |
|  0.033482909202575684   |
|  0.002450899686664343   |
|  0.016357386484742165   |
|  0.022138111293315887   |
|  0.009168407879769802   |
|  0.017992138862609

In [ ]:
print("Vector value:")
display(vector)


In [ ]:
len(vector)

### Document similarity modeled as cosine distance

In [89]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

sentences1 = ['AI']
sentences2 = ['AI','Artificial Intelligence','AGI','Machine Learning','Deep Learning','Azure OpenAI']

embeddings1 = get_embeddings(sentences1)
embeddings2 = get_embeddings(sentences2)

for i in range(len(sentences1)):
    for j in range(len(sentences2)):
        print(f"{sentences1[i]} \t\t {sentences2[j]} \t\t Score: {cosine_similarity(embeddings1[i], embeddings2[j]):.4f}")

AI 		 AI 		 Score: 1.0000
AI 		 Artificial Intelligence 		 Score: 0.6296
AI 		 AGI 		 Score: 0.4526
AI 		 Machine Learning 		 Score: 0.3724
AI 		 Deep Learning 		 Score: 0.3399
AI 		 Azure OpenAI 		 Score: 0.4303


In [97]:
import numpy as np
import pandas as pd
from tabulate import tabulate


# Assuming the definition of your cosine_similarity function is as provided
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

sentences1 = ['ข้าวต้มกุ้ง']
sentences2 = ['ข้าวต้มกุ้ง','ข้ามต้มกุ้ง','ข้าวผัดปู','เคอร์รี่ไก่กับข้าวบัสมาติ']

# Calculate cosine similarity scores
scores = []
for i in range(len(sentences1)):
    for j in range(len(sentences2)):
        score = cosine_similarity(embeddings1[i], embeddings2[j])
        scores.append([sentences1[i], sentences2[j], score])

# Create a DataFrame
df = pd.DataFrame(scores, columns=['Query word', 'Target word', 'Cosine Similarity'])

# Print the table beautifully using tabulate
print(tabulate(df, headers='keys', tablefmt='pretty', showindex=False, numalign="right", floatfmt=".4f"))

+------------+-------------------+---------------------+
| Query word |    Target word    |  Cosine Similarity  |
+------------+-------------------+---------------------+
|  ข้าวต้มกุ้ง   |      ข้าวต้มกุ้ง      | 0.9999999999999998  |
|  ข้าวต้มกุ้ง   |      ข้ามต้มกุ้ง      | 0.6295935961283562  |
|  ข้าวต้มกุ้ง   |      ข้างต้มกุ้ง      | 0.45263174308235454 |
|  ข้าวต้มกุ้ง   |      ข้าวผัดปู       | 0.37238151412401393 |
|  ข้าวต้มกุ้ง   | เคอร์รี่ไก่กับข้าวบัสมาติ | 0.3399210870922291  |
+------------+-------------------+---------------------+


In [ ]:
sentences1 = ['ชาบูชาบู (Shabu-shabu) - อาหารญี่ปุ่นที่ใช้น้ำซุปร้อนๆ ในการจิ้มเนื้อสัตว์และผักสด มีลักษณะการรับประทานที่ผู้คนรอบโต๊ะสามารถปรุงอาหารได้ด้วยตนเองเหมือนหม้อไฟ.']

sentences2 = [
    'ชาบูชาบู',
    'สุกี้ยากี้ (Sukiyaki) - เป็นอาหารญี่ปุ่นที่มีวิธีการปรุงคล้ายกับชาบูชาบู แต่สุกี้ยากี้จะมีน้ำซุปหวานและเนื้อสัตว์จะถูกจิ้มลงในไข่ดิบก่อนทาน ซึ่งให้ประสบการณ์การรับประทานที่คล้ายคลึงกับหม้อไฟ.',
    'จิ้มจุ่ม - อาหารไทยที่ใช้น้ำซุปร้อนเพื่อต้มเนื้อสัตว์และผัก ให้ประสบการณ์การรับประทานที่คล้ายกับหม้อไฟ โดยมีการใช้น้ำจิ้มรสเข้มข้นเพื่อเพิ่มรสชาติ.'
]

# Calculate cosine similarity scores
scores = []
for i in range(len(sentences1)):
    for j in range(len(sentences2)):
        score = cosine_similarity(embeddings1[i], embeddings2[j])
        scores.append([sentences1[i], sentences2[j], score])

# Create a DataFrame
df = pd.DataFrame(scores, columns=['Sentence 1', 'Sentence 2', 'Cosine Similarity'])

# Print the table beautifully using tabulate
print(tabulate(df, headers='keys', tablefmt='pretty', showindex=False, numalign="right", floatfmt=".4f"))

In [ ]:
import pandas as pd
# FILEPATH: /workspaces/vector-search-demos/vector_embeddings.ipynb
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

sentences1 = ['ชาบูชาบู (Shabu-shabu) - อาหารญี่ปุ่นที่ใช้น้ำซุปร้อนๆ ในการจิ้มเนื้อสัตว์และผักสด มีลักษณะการรับประทานที่ผู้คนรอบโต๊ะสามารถปรุงอาหารได้ด้วยตนเองเหมือนหม้อไฟ.']

sentences2 = [
    'ชาบูชาบู',
    'สุกี้ยากี้ (Sukiyaki) - เป็นอาหารญี่ปุ่นที่มีวิธีการปรุงคล้ายกับชาบูชาบู แต่สุกี้ยากี้จะมีน้ำซุปหวานและเนื้อสัตว์จะถูกจิ้มลงในไข่ดิบก่อนทาน ซึ่งให้ประสบการณ์การรับประทานที่คล้ายคลึงกับหม้อไฟ.',
    'จิ้มจุ่ม - อาหารไทยที่ใช้น้ำซุปร้อนเพื่อต้มเนื้อสัตว์และผัก ให้ประสบการณ์การรับประทานที่คล้ายกับหม้อไฟ โดยมีการใช้น้ำจิ้มรสเข้มข้นเพื่อเพิ่มรสชาติ.'
]

embeddings1 = get_embeddings(sentences1)
embeddings2 = get_embeddings(sentences2)

scores = []
for i in range(len(sentences1)):
    for j in range(len(sentences2)):
        score = cosine_similarity(embeddings1[i], embeddings2[j])
        scores.append([sentences1[i], sentences2[j], score])

df = pd.DataFrame(scores, columns=['Sentence 1', 'Sentence 2', 'Score'])
print(df)


### Vector search

In [ ]:
# Compute vector for query
query = "ชาบูชาบู (Shabu-shabu) - อาหารญี่ปุ่นที่ใช้น้ำซุปร้อนๆ ในการจิ้มเนื้อสัตว์และผักสด มีลักษณะการรับประทานที่ผู้คนรอบโต๊ะสามารถปรุงอาหารได้ด้วยตนเองเหมือนหม้อไฟ."

embeddings_response = openai_client.embeddings.create(model=AZURE_OPENAI_ADA_DEPLOYMENT, input=[query])
vector = embeddings_response.data[0].embedding

# Compute embeddings for food names
food_names = ['ชาบูชาบู', 
              'สุกี้ยากี้ (Sukiyaki) - เป็นอาหารญี่ปุ่นที่มีวิธีการปรุงคล้ายกับชาบูชาบู แต่สุกี้ยากี้จะมีน้ำซุปหวานและเนื้อสัตว์จะถูกจิ้มลงในไข่ดิบก่อนทาน ซึ่งให้ประสบการณ์การรับประทานที่คล้ายคลึงกับหม้อไฟ', 
              'จิ้มจุ่ม - อาหารไทยที่ใช้น้ำซุปร้อนเพื่อต้มเนื้อสัตว์และผัก ให้ประสบการณ์การรับประทานที่คล้ายกับหม้อไฟ โดยมีการใช้น้ำจิ้มรสเข้มข้นเพื่อเพิ่มรสชาติ.', 
              'อาหารสุนัข', 
              'อเมริกา']
food_vectors = get_embeddings(food_names)

# Compute cosine similarity between query and each food name
scores = []
for i in range(len(food_names)):
    scores.append((food_names[i], cosine_similarity(vector, food_vectors[i])))

# Display the top 10 results
df = pd.DataFrame(scores, columns=['Food Name', 'Score'])
df = df.sort_values('Score', ascending=False)
df.head(10)
